|   |  SP =  64; k = 50   |  SP = 96; k = 65     |  SP = 128; k = 80    |
|---|--------------|--------------|--------------|
|   | (a1) T = 77824        | (b1)  T = 76800       | (c1) T = 80256        |
|   | (a2) T = 38912        | (b2)  T = 38400       | (c2) T = 40128        |
|   | (a3) T = 19456        | (b3)  T = 18432       | (c3) T = 20064         |

In [2]:
from symbolic_variables import *
from gen_params import gen_ring_param, gen_sis_param, gen_sis_param_aggresive
from ring_params import *
from protocol import Protocol
from util import pretty_size
import sys
sys.path.insert(1, '../lattice-estimator')
from estimator import *


log_q = 62
f = 24
C = 3
f_hat = f / 2
q = 4611686019232694273
degree = 8
degree_vdf = 4

ring_params = {'f': f,
 'fhat': f_hat,
 'phi': euler_phi(f),
 'q': q,
 'log_q': log_q,
 'Rq_size': euler_phi(f) * log_q ,
 'e': 2,
 'C': C,
 'gamma_2': 2,
 'theta_2': 2 * f,
 'gamma_inf': 3,
 'theta_inf': 2 * f,
 'ring_exp_inf': f}


In [3]:
def evaluate():
    ring_param = ring_params
    
    sis_param = {'n': module_cyclotomic, 'log_beta_sis': log_beta_sis}
    wit_rdim = log_q * t * module_vdf
    beta_inf = 2 
    beta_2 = sqrt(wit_rdim * ring_param["phi"] * ring_param["fhat"]) * beta_inf
    protocol = Protocol(
        ring_param = ring_param,
        sis_param = sis_param,
        rep = rep, 
        wit_rdim = wit_rdim,  
        beta_2 = beta_2,
        beta_inf = beta_inf,
        # The first `norm` should be viewed as `norm+bin` check combined.
        # Such a check would increase nbot by 4 instead of 3. 
        # We compensate for it by setting the initial nbot as 2 instead of 1. 
        nbot=2
    )
    for op, params in ops:
        protocol.execute(op, **params)
    protocol.extract()
    
    print("init size: " + pretty_size(protocol.history[0].wit_size / 2))
    
    # Y_bottom size
    size_1 = (rep * module_cyclotomic * ring_param["phi"] * log_q)
    # Y_top size
    size_2 = ((rep + 1) * sis_param['n'] * ring_param["phi"] * log_q)
    
    final_wit_size = protocol.history[len(protocol.history) - 1].prover_comm
    
    print("comm size: " + pretty_size(size_1 + size_2 + final_wit_size))
    total_t = t * rep
    print("T: ", total_t)
    
    # protocol.print()
    
    params = SIS.Parameters(n=module_cyclotomic * degree, m=module_cyclotomic * degree * log_q, q=q, length_bound=2^log_beta_sis, norm=2)
    res = SIS.estimate(params)
    rop = float((str(res).split(' ')[2].split('^')[1].split(',')[0]))
    assert rop > secpar
    
    params = SIS.Parameters(n=module_vdf * degree_vdf, m=module_vdf * degree_vdf * log_q, q=q, length_bound=2 * protocol.history[0].beta_ext_2, norm=2)
    res = SIS.estimate(params)
    rop = float((str(res).split(' ')[2].split('^')[1].split(',')[0]))
    assert rop > secpar
    
    for h in protocol.history:
        assert h.snderr < 2^(-kappa)

    foldable_structure_len = 2^len((list(filter(lambda x: str(x) == 'fold', protocol.history))))
    # assert block_len % module_vdf == 0
    # assert block_len % module_cyclotomic == 0
    assert t % foldable_structure_len == 0

    #print(float(log(2 * protocol.history[0].beta_ext_2, 2)))

    finish = protocol.history[len(protocol.history) - 1]
    
    print(str(protocol.history[len(protocol.history) - 2].nbot + module_cyclotomic) +  ",")
    print(str(protocol.history[len(protocol.history) - 2].rep) +  ",")
    print(str(finish.wit_rdim) +  ",")
    print(str(round(float(log(finish.beta_ext_2) / log(2)),1)) +  ",")

    
    print("delta: " + str(round(float(total_t * (2^(-secpar) + module_vdf * degree_vdf * (q - 2^log_q) / q)), 4)))

    # print('''\
    #     } else if #[cfg(feature = "%s")] {
    #         pub static MODULE_SIZE: usize = %s;
    #         pub static COMMITMENT_MODULE_SIZE: usize = %s;
    #         pub static CHUNKS: usize = %s;
    #         pub static TIME: usize = %s;
    #       ''' %(mode, module_vdf, module_cyclotomic, rep, total_t));
        

In [3]:
mode = "c1"

secpar = 128
kappa = 80
tdim = 2
base = 21


log_beta_sis = 50.8
rep = 57
t = 64 * 22
module_cyclotomic = 203
module_vdf = 42

opener = [("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"base":base}), ("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener * 3 + loop + opener + loop +  [("finish", {})]

evaluate()


init size: 199.3 MB
comm size: 26.38 MB
T:  80256
lattice  :: rop: ≈2^128.1, red: ≈2^128.1, δ: 1.004452, β: 340, d: 3964, tag: euclidean
lattice  :: rop: ≈2^128.7, red: ≈2^128.7, δ: 1.004373, β: 349, d: 1286, tag: euclidean
223,
57,
57288,
30.4,
delta: 0.0024


In [4]:
mode = "c2"
secpar = 128
kappa = 80
tdim = 2
base = 21


log_beta_sis = 43.4
rep = 57
t = 32 * 22
module_cyclotomic = 149
module_vdf = 40


opener = [("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"base":base}), ("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener * 2 + loop + opener + loop +  [("finish", {})]

evaluate()

init size: 94.91 MB
comm size: 22.15 MB
T:  40128
lattice  :: rop: ≈2^128.7, red: ≈2^128.7, δ: 1.004425, β: 343, d: 3405, tag: euclidean
lattice  :: rop: ≈2^131.4, red: ≈2^131.4, δ: 1.004290, β: 359, d: 1266, tag: euclidean
166,
57,
54560,
30.3,
delta: 0.0011


In [5]:
mode = "c3"
secpar = 128
kappa = 80
tdim = 2
base = 21


log_beta_sis = 42.9
rep = 57
t = 32 * 11
module_cyclotomic = 145
module_vdf = 37



opener = [("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"base":base}), ("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener * 2 + loop + opener + loop +  [("finish", {})]

evaluate()

init size: 43.89 MB
comm size: 14.01 MB
T:  20064
lattice  :: rop: ≈2^128.1, red: ≈2^128.1, δ: 1.004443, β: 341, d: 3352, tag: euclidean
lattice  :: rop: ≈2^130.5, red: ≈2^130.5, δ: 1.004315, β: 356, d: 1215, tag: euclidean
162,
57,
25234,
29.8,
delta: 0.0005


In [6]:
mode = "b1"

secpar = 96
kappa = 65
tdim = 2
base = 21


log_beta_sis = 49.9
rep = 48
t = 64 * 25
module_cyclotomic = 150
module_vdf = 32

opener = [("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"base":base}), ("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener * 3 + loop + opener + loop +  [("finish", {})]

evaluate()

init size: 145.3 MB
comm size: 18.55 MB
T:  76800
lattice  :: rop: ≈2^96.5, red: ≈2^96.5, δ: 1.005804, β: 227, d: 2981, tag: euclidean
lattice  :: rop: ≈2^97.1, red: ≈2^97.1, δ: 1.005663, β: 236, d: 986, tag: euclidean
170,
48,
49600,
29.8,
delta: 0.0017


In [7]:
mode = "b2"

secpar = 96
kappa = 65
tdim = 2
base = 21


log_beta_sis = 42.7
rep = 48
t = 32 * 25
module_cyclotomic = 110
module_vdf = 30

opener = [("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"base":base}), ("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener * 2 + loop + opener + loop +  [("finish", {})]

evaluate()




init size: 68.12 MB
comm size: 15.53 MB
T:  38400
lattice  :: rop: ≈2^96.2, red: ≈2^96.2, δ: 1.005804, β: 227, d: 2555, tag: euclidean
lattice  :: rop: ≈2^97.3, red: ≈2^97.3, δ: 1.005647, β: 237, d: 957, tag: euclidean
127,
48,
46500,
29.8,
delta: 0.0008


In [8]:
mode = "b3"

secpar = 96
kappa = 65
tdim = 2
base = 21


log_beta_sis = 42.2
rep = 48
t = 32 * 12
module_cyclotomic = 108
module_vdf = 28

opener = [("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"base":base}), ("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener * 2 + loop + opener + loop +  [("finish", {})]


evaluate()



init size: 30.52 MB
comm size: 9.581 MB
T:  18432
lattice  :: rop: ≈2^96.8, red: ≈2^96.8, δ: 1.005772, β: 229, d: 2538, tag: euclidean
lattice  :: rop: ≈2^98.0, red: ≈2^98.0, δ: 1.005602, β: 240, d: 927, tag: euclidean
125,
48,
20832,
29.2,
delta: 0.0004


In [121]:
mode = "a0"

secpar = 64
kappa = 50
tdim = 2
base = 21

log_beta_sis = 49.3
rep = 38
t = 64 * 64
module_cyclotomic = 98
module_vdf = 23

opener = [("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"base":base}), ("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener * 3 + loop + opener + loop +  [("finish", {})]

evaluate()

print("-------")

mode = "a0-3"

secpar = 64
kappa = 50
tdim = 2
base = 21

log_beta_sis = 52.1
rep = 38
t = 64 * 64
module_cyclotomic = 109
module_vdf = 23

opener = [("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"base":base}), ("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener * 3 + loop + opener * 2 + loop + [("finish", {})]

evaluate()

print("-------")

mode = "a0-2"

secpar = 64
kappa = 50
tdim = 2
base = 21

log_beta_sis = 59.4
rep = 38
t = 64 * 64
module_cyclotomic = 141
module_vdf = 23

opener = [("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"base":base}), ("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener * 4 + loop + opener + loop + [("finish", {})]

evaluate()



init size: 211.7 MB
comm size: 20.74 MB
T:  155648
lattice  :: rop: ≈2^64.5, red: ≈2^64.5, δ: 1.008705, β: 113, d: 1971, tag: euclidean
lattice  :: rop: ≈2^65.5, red: ≈2^65.5, δ: 1.008324, β: 123, d: 690, tag: euclidean
118,
38,
91264,
29.6,
delta: 0.0025
-------
init size: 211.7 MB
comm size: 14.05 MB
T:  155648
lattice  :: rop: ≈2^64.6, red: ≈2^64.6, δ: 1.008705, β: 113, d: 2075, tag: euclidean
lattice  :: rop: ≈2^65.5, red: ≈2^65.5, δ: 1.008324, β: 123, d: 690, tag: euclidean
132,
38,
45632,
29.9,
delta: 0.0025
-------
init size: 211.7 MB
comm size: 14.54 MB
T:  155648
lattice  :: rop: ≈2^64.5, red: ≈2^64.5, δ: 1.008745, β: 112, d: 2354, tag: euclidean
lattice  :: rop: ≈2^65.5, red: ≈2^65.5, δ: 1.008324, β: 123, d: 690, tag: euclidean
164,
38,
45632,
29.1,
delta: 0.0025


In [9]:
mode = "a1"

secpar = 64
kappa = 50
tdim = 2
base = 21

log_beta_sis = 48.8
rep = 38
t = 64 * 32
module_cyclotomic = 96
module_vdf = 21

opener = [("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"base":base}), ("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener * 3 + loop + opener + loop +  [("finish", {})]

evaluate()


init size: 96.63 MB
comm size: 11.68 MB
T:  77824
lattice  :: rop: ≈2^64.5, red: ≈2^64.5, δ: 1.008705, β: 113, d: 1951, tag: euclidean
lattice  :: rop: ≈2^64.1, red: ≈2^64.1, δ: 1.008510, β: 118, d: 652, tag: euclidean
116,
38,
41664,
29.0,
delta: 0.0011


In [10]:
mode = "a2"

secpar = 64
kappa = 50
tdim = 2
base = 21


log_beta_sis = 41.8
rep = 38
t = 32 * 32
module_cyclotomic = 71
module_vdf = 20

opener = [("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"base":base}), ("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener * 2 + loop + opener + loop +  [("finish", {})]

evaluate()


init size: 46.02 MB
comm size: 10.04 MB
T:  38912
lattice  :: rop: ≈2^64.8, red: ≈2^64.8, δ: 1.008626, β: 115, d: 1684, tag: euclidean
lattice  :: rop: ≈2^65.2, red: ≈2^65.2, δ: 1.008361, β: 122, d: 642, tag: euclidean
88,
38,
39680,
29.0,
delta: 0.0005


In [11]:
mode = "a3"

secpar = 64
kappa = 50
tdim = 2
base = 21


log_beta_sis = 41.3
rep = 38
t = 32 * 16
module_cyclotomic = 69
module_vdf = 19

opener = [("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
loop = [("bdecomp", {"base":base}), ("norm", {}), ("split", {"tdim":tdim}), ("fold", {"repout":rep})]
ops = opener * 2 + loop + opener + loop +  [("finish", {})]

evaluate()


init size: 21.86 MB
comm size: 6.203 MB
T:  19456
lattice  :: rop: ≈2^64.5, red: ≈2^64.5, δ: 1.008665, β: 114, d: 1657, tag: euclidean
lattice  :: rop: ≈2^66.5, red: ≈2^66.5, δ: 1.008182, β: 127, d: 632, tag: euclidean
86,
38,
18848,
28.5,
delta: 0.0003
